In [1]:
import pandas as pd
poly_data = pd.read_excel('resulting_dataset.xlsx')

In [2]:
poly_data_cols_medians = [a for a in poly_data.columns if 'median' in a]
poly_data_cols_vars = [a for a in poly_data.columns if 'variance' in a]

In [3]:
new_df = pd.read_csv('cols.csv')

In [8]:
vary = poly_data[poly_data_cols_vars]
var

,Thermal conductivity_value_variance,Dynamic mechanical properties loss tangent_value_variance,Dynamic compressive properties loss tangent_value_variance,Thermal decomposition temperature_value_variance,Compressive stress strength at break_value_variance,Hansen parameter delta-h: hydrogen bonding_value_variance,Radiation resistance_value_variance,Flexural modulus_value_variance,Compressive modulus_value_variance,Vicat softening temperature_value_variance,...,Dielectric constant ac_value_variance,Fiber tensile modulus_value_variance,Shear stress strength at break_value_variance,Crystallization kinetics r_value_variance,Contact angle_value_variance,Surface tension_value_variance,Interfacial tension_value_variance,Specific volume_value_variance,Tensile creep compliance_value_variance,Tensile creep rupture strength_value_variance
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,True,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
2,False,True,True,True,True,True,True,True,True,False,...,False,True,True,False,True,False,False,False,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,False,False,True,True
4,True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18307,True,True,True,True,True,True,True,True,True,True,...,False,True,True,True,True,True,True,True,True,True
18308,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
18309,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,True
18310,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [16]:
nonzero_counts = pd.DataFrame({
   # 'var': vary.columns,
    'count': 18312 - vary.apply(lambda col: col.astype(bool).sum())
})


In [18]:
nonzero_counts.to_excel('non_zero_var.xlsx')

In [23]:
vary['Volume resistivity_value_variance']#.isna().sum()

0        3354844969133619982821830547268557602816
1           1620381465120619970405709793344880640
2                                                
3                                                
4                                                
                           ...                   
18307                                            
18308                                            
18309                            1514753521.55152
18310                                            
18311                                            
Name: Volume resistivity_value_variance, Length: 18312, dtype: object

## Joblib

In [46]:
import joblib
from sklearn.ensemble import RandomForestRegressor

# Training
def train_random_forest(X_train, y_train):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    return model

# Save the model
model = train_random_forest(X_train, y_train)
joblib.dump(model, 'model.joblib')

# Load the model
loaded_model = joblib.load('model.joblib')

# Prediction
y_pred = loaded_model.predict(X_test)


In [47]:
loaded_model.predict(X_null[cols])

array([315.345 , 345.1065, 344.9615, ..., 416.2605, 344.895 , 369.051 ])

In [50]:
loaded_model = joblib.load(r'C:\Users\ivan\SCIENSE\Polymers\saved models\Gradient Boosting\Thermal conductivity_Gradient Boosting_model.joblib')

In [59]:
y_null = loaded_model.predict(X_null[cols])
y_null

array([0.19407939, 0.19407939, 0.20254395, ..., 0.46068163, 0.54953813,
       0.5238544 ])

In [83]:
characteristic_name = 'Thermal conductivity'
regressor_name = 'Gradient Boosting'

model = joblib.load(fr'saved models\{regressor_name}\{characteristic_name}_{regressor_name}_model.joblib')

# Get X_null (assuming it's already defined)
cols = [c for c in new_df.columns if 'col' in c]

new_df[f'{characteristic_name}'] = poly_data[f'{characteristic_name}_value_median']
new_df[f'polymer_name'] = poly_data[f'polymer_name']
new_df = new_df[new_df[f'{characteristic_name}'].isna() == True]#.drop([f'{characteristic_name}'], axis=1)
X_null = new_df[cols]

y_null = model.predict(X_null)
new_df[f'{characteristic_name}'] = list(y_null)
meta_df = new_df[['polymer_name', f'{characteristic_name}']]

meta_df.to_excel(f'{characteristic_name}_{regressor_name}_null_predictions.xlsx')

In [82]:
meta_df

,polymer_name,Thermal conductivity
3,poly(pent-1-ene),0.194079
4,poly(3-methylbut-1-ene),0.194079
5,poly(hex-1-ene),0.202544
7,poly(hept-1-ene),0.202544
8,poly(5-methylhex-1-ene),0.333966
...,...,...
18307,"poly{2,3-diphenyl-6-[6,7-diphenylnaphthalene-2...",0.925377
18308,"poly(1,1,2-trifluoro-2-{1,1,2,4,4,5,5-heptaflu...",0.176871
18309,"poly(naphthalene-4,5:8,1-tetrayl)",0.460682
18310,poly((phenylarsanediyl){1-[(phenylarsanediyl)e...,0.549538


## Объединение

In [20]:
data = poly_data[poly_data_cols_medians]

C:\Users\ivan\AppData\Local\Temp\ipykernel_15540\1879485668.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['polymer_name'] = poly_data['polymer_name']


In [23]:
# Assuming df is your DataFrame
data.columns = data.columns.str.replace('_value_median', '')


In [27]:
bests = pd.read_excel('max_r2.xlsx')
best_chars = list(bests['Charactristic'])

In [29]:
data = data[best_chars]
data['polymer_name'] = poly_data['polymer_name']

C:\Users\ivan\AppData\Local\Temp\ipykernel_15540\893231387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['polymer_name'] = poly_data['polymer_name']


In [4]:
data = pd.read_excel('for_filling.xlsx')
data

,Unnamed: 0,Thermal conductivity,Dynamic mechanical properties loss tangent,Thermal decomposition temperature,Hansen parameter delta-h: hydrogen bonding,Flexural modulus,Vicat softening temperature,Dynamic mechanical properties loss modulus,Pvt relation specific volume,Tensile modulus,...,Second virial coefficient,Hansen parameter delta-d: dispersion component,Softening temperature,Dielectric constant ac,Fiber tensile modulus,Crystallization kinetics r,Contact angle,Surface tension,Specific volume,polymer_name
0,0,0.330,0.127,435.0,0.0,0.598000,123.0,0.150,1.1270,0.7386,...,0.000286,0.0,96.0,2.30,21.5,79175.0,53.0,31.00,1.073,polyethene
1,1,0.210,0.055,373.0,NaN,1.364355,138.0,0.086,1.1737,1.1200,...,0.000560,NaN,27.5,2.20,NaN,215.0,61.5,29.00,1.106,poly(prop-1-ene)
2,2,0.218,NaN,NaN,NaN,NaN,99.0,0.140,1.1358,0.2089,...,NaN,NaN,NaN,2.32,NaN,70.0,NaN,26.40,1.100,poly(but-1-ene)
3,3,NaN,NaN,NaN,NaN,NaN,NaN,0.220,NaN,NaN,...,NaN,NaN,-29.0,NaN,NaN,NaN,NaN,37.07,1.100,poly(pent-1-ene)
4,4,NaN,NaN,452.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,1.100,poly(3-methylbut-1-ene)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18307,18307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.60,NaN,NaN,NaN,NaN,NaN,"poly{2,3-diphenyl-6-[6,7-diphenylnaphthalene-2..."
18308,18308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"poly(1,1,2-trifluoro-2-{1,1,2,4,4,5,5-heptaflu..."
18309,18309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621,"poly(naphthalene-4,5:8,1-tetrayl)"
18310,18310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,poly((phenylarsanediyl){1-[(phenylarsanediyl)e...


In [33]:
import pandas as pd

# Load the DataFrames
#df = pd.read_csv('your_original_data.csv')  # Replace with your actual file path or method of loading data
null_predictions_df = pd.read_excel('predicts/Specific volume_XGBoost_null_predictions.xlsx')

# Replace None values in 'Specific volume' column in df with corresponding values from null_predictions_df
data['Specific volume'] = data['Specific volume'].combine_first(null_predictions_df.set_index('polymer_name')['Specific volume'])

# Now, df['Specific volume'] should have None values filled with corresponding values from null_predictions_df


In [34]:
data

,Unnamed: 0,Thermal conductivity,Dynamic mechanical properties loss tangent,Thermal decomposition temperature,Hansen parameter delta-h: hydrogen bonding,Flexural modulus,Vicat softening temperature,Dynamic mechanical properties loss modulus,Pvt relation specific volume,Tensile modulus,...,Second virial coefficient,Hansen parameter delta-d: dispersion component,Softening temperature,Dielectric constant ac,Fiber tensile modulus,Crystallization kinetics r,Contact angle,Surface tension,Specific volume,polymer_name
0,0,0.330,0.127,435.0,0.0,0.598000,123.0,0.150,1.1270,0.7386,...,0.000286,0.0,96.0,2.30,21.5,79175.0,53.0,31.00,1.073,polyethene
1,1,0.210,0.055,373.0,NaN,1.364355,138.0,0.086,1.1737,1.1200,...,0.000560,NaN,27.5,2.20,NaN,215.0,61.5,29.00,1.106,poly(prop-1-ene)
2,2,0.218,NaN,NaN,NaN,NaN,99.0,0.140,1.1358,0.2089,...,NaN,NaN,NaN,2.32,NaN,70.0,NaN,26.40,1.100,poly(but-1-ene)
3,3,NaN,NaN,NaN,NaN,NaN,NaN,0.220,NaN,NaN,...,NaN,NaN,-29.0,NaN,NaN,NaN,NaN,37.07,1.100,poly(pent-1-ene)
4,4,NaN,NaN,452.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,1.100,poly(3-methylbut-1-ene)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18307,18307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.60,NaN,NaN,NaN,NaN,NaN,"poly{2,3-diphenyl-6-[6,7-diphenylnaphthalene-2..."
18308,18308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"poly(1,1,2-trifluoro-2-{1,1,2,4,4,5,5-heptaflu..."
18309,18309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621,"poly(naphthalene-4,5:8,1-tetrayl)"
18310,18310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,poly((phenylarsanediyl){1-[(phenylarsanediyl)e...


In [14]:
# Assuming df_data is your original DataFrame and df_null_predictions is the one with null predictions
# Assuming 'polymer_name' is the common key column
df_data = data

char = 'Specific volume'
# Load the null predictions DataFrame
df_null_predictions = pd.read_excel(f'predicts2/{char}_XGBoost_null_predictions.xlsx')

# Merge the original DataFrame with the null predictions DataFrame on 'polymer_name'
merged_df = pd.merge(df_data, df_null_predictions[['polymer_name', f'{char}']], on='polymer_name', how='left', suffixes=('', '_null'))

# Fill None values in 'Specific volume' with the corresponding values from null predictions
merged_df[f'{char}'] = merged_df[f'{char}'].fillna(merged_df[f'{char}_null'])

# Drop the extra columns used for merging
merged_df = merged_df.drop([f'{char}_null'], axis=1)

# Update the original DataFrame with the filled values
df_data = merged_df



In [15]:
df_data

,Unnamed: 0,Thermal conductivity,Dynamic mechanical properties loss tangent,Thermal decomposition temperature,Hansen parameter delta-h: hydrogen bonding,Flexural modulus,Vicat softening temperature,Dynamic mechanical properties loss modulus,Pvt relation specific volume,Tensile modulus,...,Second virial coefficient,Hansen parameter delta-d: dispersion component,Softening temperature,Dielectric constant ac,Fiber tensile modulus,Crystallization kinetics r,Contact angle,Surface tension,Specific volume,polymer_name
0,0,0.330,0.127,435.0,0.0,0.598000,123.0,0.150,1.1270,0.7386,...,0.000286,0.0,96.0,2.30,21.5,79175.0,53.0,31.00,1.073000,polyethene
1,1,0.210,0.055,373.0,NaN,1.364355,138.0,0.086,1.1737,1.1200,...,0.000560,NaN,27.5,2.20,NaN,215.0,61.5,29.00,1.106000,poly(prop-1-ene)
2,2,0.218,NaN,NaN,NaN,NaN,99.0,0.140,1.1358,0.2089,...,NaN,NaN,NaN,2.32,NaN,70.0,NaN,26.40,1.100000,poly(but-1-ene)
3,3,NaN,NaN,NaN,NaN,NaN,NaN,0.220,NaN,NaN,...,NaN,NaN,-29.0,NaN,NaN,NaN,NaN,37.07,1.100000,poly(pent-1-ene)
4,4,NaN,NaN,452.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,1.100000,poly(3-methylbut-1-ene)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18307,18307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.60,NaN,NaN,NaN,NaN,0.914352,"poly{2,3-diphenyl-6-[6,7-diphenylnaphthalene-2..."
18308,18308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600516,"poly(1,1,2-trifluoro-2-{1,1,2,4,4,5,5-heptaflu..."
18309,18309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621000,"poly(naphthalene-4,5:8,1-tetrayl)"
18310,18310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.794313,poly((phenylarsanediyl){1-[(phenylarsanediyl)e...


In [27]:
import os
import pandas as pd

# Замените 'путь_к_папке' на путь к вашей папке с файлами .xlsx
folder_path = 'resulting_files'

# Получаем список файлов в папке
file_list = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# Создаем пустой список для хранения частей названий файлов
parts_list = []

# Обходим каждый файл
for file_name in file_list:
    # Используем '_' как разделитель и берем часть до первого '_'
    parts = file_name.split('_')[0]
    parts_list.append(parts)

# Выводим список частей названий файлов
print(parts_list)



['Brittleness temperature', 'Cohesive energy density', 'Contact angle', 'Crystallization kinetics half time of crystallization', 'Crystallization kinetics k', 'Crystallization kinetics n', 'Crystallization kinetics r', 'Crystallization temperature', 'Deflection temperature under load hdt', 'Density', 'Dielectric constant ac', 'Dielectric loss factor', 'Dielectric loss tangent', 'Dynamic flexural properties loss tanget', 'Dynamic flexural properties storage modulus', 'Dynamic mechanical properties loss modulus', 'Dynamic mechanical properties loss tangent', 'Dynamic mechanical properties storage modulus', 'Dynamic shear properties loss modulus', 'Dynamic shear properties loss tangent', 'Dynamic shear properties storage modulus', 'Electric conductivity', 'Elongation at break', 'Elongation at yield', 'Fiber tensile elongation at break', 'Fiber tensile modulus', 'Fiber tensile stress strength at break', 'Flexural modulus', 'Flexural stress strength at break', 'Gas diffusion coefficient d',

In [28]:
len(parts_list)

62

In [29]:
ps = df_data.drop(['Unnamed: 0', 'polymer_name'], axis=1)
all_cols = list(ps.columns)

In [30]:
len(all_cols)

68

In [31]:
missing_names = set(all_cols) - set(parts_list)

# Преобразуем результат обратно в список, если это необходимо
missing_names_list = list(missing_names)

In [26]:
missing_names_list1 = missing_names_list

In [33]:
missing_names_list = ['Hansen parameter delta-h: hydrogen bonding',
 'Solubility parameter',
 'Vicat softening temperature',
 'Thermal decomposition temperature',
 'Surface tension']

In [6]:
import os
import pandas as pd

# Assuming df_data is your original DataFrame
df_data = data

# Specify the folder containing null prediction files
null_predictions_folder = 'predicts2'

# Specify the folder to save the resulting files
result_folder = 'resulting_files'

# Create the result folder if it doesn't exist
os.makedirs(result_folder, exist_ok=True)

# Iterate through files in the folder
for filename in os.listdir(null_predictions_folder):
    if os.path.splitext(filename)[1] == ".xlsx":
        # Extract the characteristic name from the filename (before the first '_')
        char = filename.split('_')[0]

        # Construct the full path to the file
        file_path = os.path.join(null_predictions_folder, filename)

        try:
            # Load the null predictions DataFrame
            df_null_predictions = pd.read_excel(file_path)

            # Merge the original DataFrame with the null predictions DataFrame on 'polymer_name'
            merged_df = pd.merge(df_data, df_null_predictions[['polymer_name', char]], on='polymer_name', how='left', suffixes=('', '_null'))

            # Fill None values in the characteristic column with the corresponding values from null predictions
            merged_df[char] = merged_df[char].fillna(merged_df[f'{char}_null'])

            # Drop the extra columns used for merging
            merged_df = merged_df.drop([f'{char}_null'], axis=1)

            # Update the original DataFrame with the filled values
            df_data = merged_df

            # Log success message
            print(f"Successfully processed file: {file_path}")

            # Save the resulting DataFrame to a new Excel file
            result_filename = f"{char}_result.xlsx"
            result_path = os.path.join(result_folder, result_filename)
            df_data.to_excel(result_path, index=False)

            # Log success message for saving the result
            print(f"Result saved to: {result_path}")

        except (pd.errors.EmptyDataError, pd.errors.ParserError) as e:
            # Log error message
            print(f"Error processing file {file_path}: {e}")



Successfully processed file: predicts2\Brittleness temperature_KNeighborsRegressor_null_predictions.xlsx
Result saved to: resulting_files\Brittleness temperature_result.xlsx
Successfully processed file: predicts2\Cohesive energy density_XGBoost_null_predictions.xlsx
Result saved to: resulting_files\Cohesive energy density_result.xlsx
Successfully processed file: predicts2\Contact angle_KNeighborsRegressor_null_predictions.xlsx
Result saved to: resulting_files\Contact angle_result.xlsx
Successfully processed file: predicts2\Crystallization kinetics half time of crystallization_Elastic Net_null_predictions.xlsx
Result saved to: resulting_files\Crystallization kinetics half time of crystallization_result.xlsx
Successfully processed file: predicts2\Crystallization kinetics k_XGBoost_null_predictions.xlsx
Result saved to: resulting_files\Crystallization kinetics k_result.xlsx
Successfully processed file: predicts2\Crystallization kinetics n_Elastic Net_null_predictions.xlsx
Result saved to:

Successfully processed file: predicts2\Radius of gyration_AdaBoost_null_predictions.xlsx
Result saved to: resulting_files\Radius of gyration_result.xlsx
Successfully processed file: predicts2\Refractive index_XGBoost_null_predictions.xlsx
Result saved to: resulting_files\Refractive index_result.xlsx
Successfully processed file: predicts2\Second virial coefficient_KNeighborsRegressor_null_predictions.xlsx
Result saved to: resulting_files\Second virial coefficient_result.xlsx
Successfully processed file: predicts2\Softening temperature_AdaBoost_null_predictions.xlsx
Result saved to: resulting_files\Softening temperature_result.xlsx


BadZipFile: File is not a zip file

In [34]:
import os
import pandas as pd

# Assuming df_data is your original DataFrame
df_data = data

# Specify the folder containing null prediction files
null_predictions_folder = 'predicts2'

# Specify the folder to save the resulting files
result_folder = 'resulting_files'

# Create the result folder if it doesn't exist
os.makedirs(result_folder, exist_ok=True)

# Iterate through missing names
for char in missing_names_list:
    # Iterate through files in the folder
    for filename in os.listdir(null_predictions_folder):
        # Check if the file is an Excel file and if char is in the filename
        if filename.endswith('.xlsx') and char in filename:
            # Construct the full path to the file
            file_path = os.path.join(null_predictions_folder, filename)

            try:
                # Load the null predictions DataFrame
                df_null_predictions = pd.read_excel(file_path)

                # Merge the original DataFrame with the null predictions DataFrame on 'polymer_name'
                merged_df = pd.merge(df_data, df_null_predictions[['polymer_name', char]], on='polymer_name', how='left', suffixes=('', '_null'))

                # Fill None values in the characteristic column with the corresponding values from null predictions
                merged_df[char] = merged_df[char].fillna(merged_df[f'{char}_null'])

                # Drop the extra columns used for merging
                merged_df = merged_df.drop([f'{char}_null'], axis=1)

                # Update the original DataFrame with the filled values
                df_data = merged_df

                # Log success message
                print(f"Successfully processed file: {file_path}")

                # Save the resulting DataFrame to a new Excel file
                result_filename = f"{char}_result.xlsx"
                result_path = os.path.join(result_folder, result_filename)
                df_data.to_excel(result_path, index=False)

                # Log success message for saving the result
                print(f"Result saved to: {result_path}")

            except (pd.errors.EmptyDataError, pd.errors.ParserError) as e:
                # Log error message
                print(f"Error processing file {file_path}: {e}")


Successfully processed file: predicts2\Solubility parameter_XGBoost_null_predictions.xlsx
Result saved to: resulting_files\Solubility parameter_result.xlsx
Successfully processed file: predicts2\Vicat softening temperature_Gradient Boosting_null_predictions.xlsx
Result saved to: resulting_files\Vicat softening temperature_result.xlsx
Successfully processed file: predicts2\Thermal decomposition temperature_Random Forest_null_predictions.xlsx
Result saved to: resulting_files\Thermal decomposition temperature_result.xlsx
Successfully processed file: predicts2\Surface tension_Bagging_null_predictions.xlsx
Result saved to: resulting_files\Surface tension_result.xlsx


In [35]:
import os
import pandas as pd

# Папка с файлами .xlsx
folder_path = 'final'

# Получаем список файлов в папке
file_list = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# Создаем пустой DataFrame для объединенных данных
merged_df = pd.DataFrame()

# Итерируем по файлам и добавляем данные в объединенный DataFrame
for filename in file_list:
    file_path = os.path.join(folder_path, filename)
    
    # Считываем данные из файла .xlsx
    df = pd.read_excel(file_path)
    
    # Определяем ненулевые колонки
    non_null_columns = df.columns[df.notnull().any()].tolist()
    
    # Добавляем ненулевые колонки в объединенный DataFrame
    merged_df = pd.concat([merged_df, df[non_null_columns]], axis=1)

# Удаляем дубликаты столбцов (если они есть)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Сохраняем объединенные данные в новый файл .xlsx
output_file_path = 'merged_result.xlsx'
merged_df.to_excel(output_file_path, index=False)

print(f"Merged result saved to: {output_file_path}")


Merged result saved to: merged_result.xlsx
